In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [2]:
def init_browser():
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
browser = init_browser()

#set url
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

#scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

#get headline and teaser text
headline = soup.find("div", class_="content_title").text
teaser = soup.find("div", class_="article_teaser_body").text

print(headline)
print(teaser)

NASA's InSight Places First Instrument on Mars
In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.


In [4]:
#set url
url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(1)

#scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

#get full-sized featured image
featured_img = soup.find("footer")
featured_img = featured_img.find("a")["data-fancybox-href"]
featured_img = "https://www.jpl.nasa.gov" + featured_img

print(featured_img)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA22911_ip.jpg


In [5]:
#set url
url="https://twitter.com/marswxreport?lang=en"
browser.visit(url)

time.sleep(1)

#scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

#get weather info
weather = soup.find("p", class_="tweet-text").text

print(weather)

Sol 2270 (2018-12-25), high -5C/23F, low -75C/-102F, pressure at 8.35 hPa, daylight 06:41-18:53


In [6]:
#set url
url="https://space-facts.com/mars/"

#scrape table with pandas
tables = pd.read_html(url)
mars_df = tables[0]
#set index
mars_df = mars_df.set_index(0)

#convert to html
mars_table = mars_df.to_html(header=False)
mars_table = mars_table.replace('\n', '')

print(mars_table)

<table border="1" class="dataframe">  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


In [7]:
#set url
url="http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

#scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

#find hemisphere titles
h3s = soup.find_all("h3")
hemispheres = [result.text[:-9] for result in h3s]

print(hemispheres)

['Cerberus Hemisphere', 'Schiaparelli Hemisphere', 'Syrtis Major Hemisphere', 'Valles Marineris Hemisphere']


In [8]:
#set empty list for hemisphere images
hemisphere_imgs = []

for hemisphere in hemispheres:
    #click hemisphere link
    browser.click_link_by_partial_text(hemisphere)

    time.sleep(1)
    
    #scrape page into soup
    html = browser.html
    soup = bs(html, 'html.parser')

    #find hemisphere images
    hemisphere_img = soup.find("div", class_="downloads")
    hemisphere_img = hemisphere_img.find("a")["href"]
    #add img to list
    hemipshere_imgs = hemisphere_imgs.append(hemisphere_img)

    #reset url
    url="http://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url)
    
print(hemisphere_imgs)

['http://web.archive.org/web/20181114182238/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'http://web.archive.org/web/20181114182242/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'http://web.archive.org/web/20181114182245/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'http://web.archive.org/web/20181114182248/http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']


In [ ]:
#create dict of hemisphere info
hemisphere_df = pd.DataFrame({"img_url": hemisphere_imgs, "title": hemispheres})
hemisphere_dict = hemisphere_df.to_dict("records")

print(hemisphere_dict)

In [ ]:
########
#set url
url = "https://www.google.com/search?q=mars"
browser.visit(url)

time.sleep(1)

#scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

#find summary
summary = soup.find("div", class_ = "SALvLe farUxc mJ2Mod")
summary = summary.find("span").text
summary

In [ ]:
#create dict of all mars info
mars_data = {
    "headline": headline, 
    "teaser": teaser,
    "feat_img": featured_img, 
    "weather": weather, 
    "facts": mars_table, 
    "hemispheres": hemisphere_dict,
    "summary": summary
}

# Close the browser after scraping
browser.quit()

for i in mars_data:
    print(i)
    print(mars_data[i])